In [28]:
from bs4 import BeautifulSoup
import requests
import re
import time
import pandas as pd
import numpy as np
import json
import os
from urllib.request import Request, urlopen
from tqdm import tqdm

# 1.2

### Part a

In [29]:
#defining the url of the first page
url = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=amazon+gift+card&_sacat=0&rt=nc&LH_Sold=1&LH_Complete=1"

#defining a new variable called page, user agent is basically telling that we are basically Mozilla. 
user_agent = {'User-agent': 'Mozilla/5.0'}
page = requests.get(url, headers=user_agent)

#creating a bs4 object "class" and reading the HTML contents stored in "page" which was requested above
soup = BeautifulSoup(page.text, 'lxml')

In [30]:
#creating a file "ebay_amazon gift cards_webpages" for storing all the pages of seacrh results that we download. But before that we are checking if the we already have one file with the same name. If so they code piece will not run
try:
    if not os.path.exists("ebay_amazon gift cards_webpage1"):
        os.makedirs("ebay_amazon gift cards_webpage1")
except:
    pass

In [31]:
#downloading the first pages of seacrh results 

response_page1 = requests.get(url, headers = user_agent)
soup_page1 = BeautifulSoup(response_page1.content, 'html.parser')
with open('ebay_amazon gift cards_webpage1/amazon_gift_card_01.htm', 'w', encoding='utf-8') as file:
    file.write(str(soup_page1))
time.sleep(10)

### Part b

In [32]:
#creating a list to contain all the url's for the 10 pages

list1 = []

#appending the product url's into list1
for i in range(1,11):
    list1.append(url + '&_pgn={}'.format(str(i))) 

In [33]:
#creating a file "ebay_amazon gift cards_webpages" for storing all the pages of seacrh results that we download

try:
    if not os.path.exists("ebay_amazon gift cards_webpages"):
        os.makedirs("ebay_amazon gift cards_webpages")
except:
    pass

In [34]:
#downloading the first 10 pages of seacrh results

j = 1
for i in tqdm(list1):
    response_pages1 = requests.get(i, headers = user_agent)
    soup_pages1 = BeautifulSoup(response_pages1.content, 'html.parser')
    with open('ebay_amazon gift cards_webpages/amazon_gift_card_{}.htm'.format(str(j)), 'w', encoding='utf-8') as file:
        file.write(str(soup_pages1))
    j = j + 1
    time.sleep(10)

100%|██████████| 10/10 [02:04<00:00, 12.47s/it]


### Part c

In [35]:
#defining a list which stores all the content present across the 10 downloaded html files

webpage_content = []

#in this piece of code we are accessing the downloaded pages which are stored in the file "bntop40_webpages"

for i in range(1 , len(list1)+1):
    with open('ebay_amazon gift cards_webpages/amazon_gift_card_{}.htm'.format(str(i)), encoding='utf-8') as link_to_page:
        soup_open = BeautifulSoup(link_to_page, 'html.parser')
        webpage_content.append(soup_open)


### Part d

In [36]:
#defining a list basically to store the scrapped content from all the pages of seacrh results

title_info = []
price_info = []
shipping_info = []

count = 1
for webpage in webpage_content:
    
    
    #identifying all the amazon gift cards in the webpage and storing them in a list
    product_list = webpage.find_all('div' , {'class' : 's-item__wrapper clearfix'})
    
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    print(f"Page {count}")
    print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
    
    count = count+1
    #accessing each product in the product list obtained from above
    for product in product_list:  
        
        
        #printing the title of the product 
        if product.find('div' , attrs = {'class' : 's-item__title'}).find('span').text == "Shop on eBay":
            pass
        else:
            sentence = product.find('div' , attrs = {'class' : 's-item__title'}).find('span').text
            title = re.sub(r"^New Listing", "", sentence)
            print(title)
            title_info.append(title)
            
        
        #printing the price of the product
        if product.find('div' , attrs = {'class' : 's-item__title'}).find('span').text == "Shop on eBay":
            pass
        else:
            price = product.find('span' , attrs = {'class' : 's-item__price'}).text
            print(price)
            price_info.append(price)
    
 
        #printing the shipping cost and if it doesn't have any information regarding shipping then printing $0
        if product.find('div' , attrs = {'class' : 's-item__title'}).find('span').text == "Shop on eBay":
            pass
        else:
            try:
                shipping = product.find('span' , attrs = {'class' : 's-item__shipping s-item__logisticsCost'}).text
                print(shipping)
                shipping_info.append(shipping)
            except:
                #if there is no shipping cost mentioned then I am printing "$0.00" for thoes cases
                print("No shipping cost mentioned")
                shipping_info.append("Free shipping")
        
        print("----------------------------------------------------------------")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Page 1
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
----------------------------------------------------------------
Amazon Gift Card $55
$54.00
Free shipping
----------------------------------------------------------------
Amazon US Gift Card $200 Value Physical/Mailed Delivery
$195.00
Free shipping
----------------------------------------------------------------
Amazon Gift Cards ($200 / New Physical Cards) **Free Shipping**
$195.00
Free shipping
----------------------------------------------------------------
Amazon $200 Gift Card, Brand New Unopened Physical Card, Free Shipping (US ONLY)
$202.50
Free shipping
----------------------------------------------------------------
Amazon $100 Gift Card, Brand New, Package Unopened
$93.05
+$5.25 shipping
-------------------------------------------------------------

In [37]:
#converting the above 3 lists into a data frame by the name "df"

df = pd.DataFrame({'Title':title_info, 'Price' : price_info, 'shipping' : shipping_info})

### Part e

In [38]:
#using regex to extract the face value from the title of the product

#defining an empty string to store the values post operation
face_value = []

for i in title_info:
    x = re.search("(.*?)([0-9]{1,4})(.*)",i)
    if x:
        face_value.append(re.sub("(.*?)([0-9]{1,4})(.*)",r"\2",i))
    else:
        face_value.append("NA")

In [39]:
#using regex to extract the price from the product

#defining an empty string to store the values post operation
price_value = []

for i in price_info:
    if i.count("$") > 1:
        price_value.append("No price value")
    else:
        price_value.append(float(re.sub("(\$)([0-9]+)(.)([0-9]+)",r"\2\3\4",i)))

In [40]:
#using regex to extract the shipping cost 

#defining an empty string to store the values post operation
shipping_value = []

for i in shipping_info:
    if i == "Free shipping":
        shipping_value.append("0.00")
    else:
        shipping_value.append(float(re.sub('(.*)([0-9]+)(.)([0-9]+)(.*)' , r"\2\3\4" ,i)))

In [41]:
#converting the above 3 lists into a data frame

df_value = pd.DataFrame({'Face Value':face_value, 'Price Value' : price_value, 'Shipping Value' : shipping_value})
df_value['Gift Card Number'] = [i for i, _ in enumerate(df_value.index)]

In [42]:
#dropping the rows which has the following strings displayed in their rows

string_to_drop_1 = 'NA'
string_to_drop_2 = 'No price value'

df_value_it1 = df_value[~df_value['Face Value'].str.contains(string_to_drop_1, na=False)]
df_value_final = df_value_it1[~df_value_it1['Price Value'].str.contains(string_to_drop_2, na=False)]

In [43]:
#converting the data types of columns to float

df_value_final['Face Value'] = df_value_final['Face Value'].astype(float)
df_value_final['Price Value'] = df_value_final['Price Value'].astype(float)
df_value_final['Shipping Value'] = df_value_final['Shipping Value'].astype(float)

C:\Users\SANKA NAGA NITESH\AppData\Local\Temp\ipykernel_32360\3453810468.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_value_final['Face Value'] = df_value_final['Face Value'].astype(float)
C:\Users\SANKA NAGA NITESH\AppData\Local\Temp\ipykernel_32360\3453810468.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_value_final['Price Value'] = df_value_final['Price Value'].astype(float)
C:\Users\SANKA NAGA NITESH\AppData\Local\Temp\ipykernel_32360\3453810468.py:5: SettingWithCopyWarning: 
A value 

In [44]:
#appending a new column to the above data frame which displays "yes" and "no" when below conditions are met

df_value_final.loc[df_value_final['Face Value'] < df_value_final['Price Value'] + df_value_final['Shipping Value'], '>face'] = '1'
df_value_final.loc[df_value_final['Face Value'] >= df_value_final['Price Value'] + df_value_final['Shipping Value'], '>face'] = '0'

C:\Users\SANKA NAGA NITESH\AppData\Local\Temp\ipykernel_32360\2758287292.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_value_final.loc[df_value_final['Face Value'] < df_value_final['Price Value'] + df_value_final['Shipping Value'], '>face'] = '1'


In [45]:
#displaying a data frame consisting of gift cards row numbers whoes face value is less than their selling price (shipping and price)

df_index = pd.DataFrame(df_value_final[df_value_final['>face'] == '1']['Gift Card Number'])

In [46]:
#displaying details of the gift cards that were sold above the face value

count = 1
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
print("Gift Cards that are sold above the Face Value")
print("~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~")
for value in df_index['Gift Card Number']:
    print(f"Gift Card {count}")
    print(df.iloc[value])
    print("----------------------------------------------------------------")
    count = count+1
print("\n")

~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Gift Cards that are sold above the Face Value
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
Gift Card 1
Title       Amazon $200 Gift Card, Brand New Unopened Phys...
Price                                                 $202.50
shipping                                        Free shipping
Name: 3, dtype: object
----------------------------------------------------------------
Gift Card 2
Title       Amazon $100 Gift Card, Brand New, Package Unop...
Price                                                  $98.00
shipping                                      +$5.25 shipping
Name: 5, dtype: object
----------------------------------------------------------------
Gift Card 3
Title       Amazon Gift Card $300 Value
Price                           $315.00
shipping                +$5.00 shipping
Name: 8, dtype: object
-

### Part e

In [47]:
#finding the fraction of gift cards that are sold above the face value among all the listed gift cards in our case

total_number_of_gift_cards = len(df)
total_number_of_overpriced_gift_cards = len(df_index)
fraction = ((total_number_of_overpriced_gift_cards / total_number_of_gift_cards)*100)
print("The fraction of gift cards that were sold above the face value are {}".format(fraction),"%")
print("\n")

print("So I can think of three reasons:")
print("  1. It could be because people are purchasing the gift cards for gifting purposes and hecen they don't mind the additional cost (which is shipping cost)")
print("  2. It could be because some of the products available on amazon might be offering you additional discount on total purchase value and hence people don't mind paying a bit more dollars over the face value.")
print("  3. For illegal purposes, money laundering ")

The fraction of gift cards that were sold above the face value are 40.33333333333333 %


So I can think of three reasons:
  1. It could be because people are purchasing the gift cards for gifting purposes and hecen they don't mind the additional cost (which is shipping cost)
  2. It could be because some of the products available on amazon might be offering you additional discount on total purchase value and hence people don't mind paying a bit more dollars over the face value.
  3. For illegal purposes, money laundering 


# 2.2

### Part a 

In [48]:
### my user name, pwd and other payload

url_login = 'https://www.fctables.com/user/login/'

payload = {
    'login_action' : '1',
    'login_username': 'jorn_the_boss',
    'login_password': '1q2w3e4r5t',
    'user_remeber' : '1',
    'submit' : '1'}

session_requests = requests.session()
res = session_requests.post(url_login, data = payload, timeout = 15)
cookies = session_requests.cookies.get_dict()

#printing the cookies
print(cookies)

{'PHPSESSID': 'g415os411fjrlm0rr6sfl8had0', 'auth_auto': '72643ecb6d3854b99f4cda470a8fde54', 'user': '126925', 'user_logged': '1', 'user_login': 'jorn_the_boss'}


### Part b

In [49]:
#url of the betting page 
url_bets = 'https://www.fctables.com/tipster/my_bets/'

session_requests = requests.session()

#getting the contents of above mentioned url into "doc"
page = session_requests.get(url_bets, cookies=cookies, timeout = 15)
doc = BeautifulSoup(page.content, 'html.parser')

In [50]:
#checking if Wolfsburg is present in doc2 or not

if "Wolfsburg" in doc.text:
    print("Yayyy! Wolfsburg is still there")
else:
    print("Check your code again!!!")


Yayyy! Wolfsburg is still there
